# dEdx weights calculation

## Input

This notebook expects to find a txt file called "VolumesZPosition_detailedsimulation.txt". This file is coming from the _Validation/Geometry_ package and more details on the setup and the procedure can be found here: 

https://twiki.cern.ch/twiki/bin/view/Sandbox/HGCalMaterialBudget
Just for the record we correct for the Materials with spaces in their name: 
sed -i -e 's/M_NEMA\ FR4\ plate/M_NEMA_FR4_plate/g' VolumesZPosition.txt

In [1]:
# Add one line at top for the EE front with Stainless Steel for the track of your interest e.g
# StainlessSteel 3190.5 0 0 0 
# or -3190.5 . Sorry for this manual change but it simplifies things until I can automate it. 
inputfile = "VolumesZPosition_detailedsimulation.txt"

## Some necessary imports

In [2]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pylab as plt
import seaborn as sns

## Materials dEdx and radiation lengths

In [3]:
#In MeV/mm
dEdx = OrderedDict()
#--------
#Some elements necessary to build our materials
dEdx['Fe'] = 1.143
dEdx['Mn'] = 1.062
dEdx['Cr'] = 1.046
dEdx['Ni'] = 1.307
dEdx['C']  = 0.3952
dEdx['0']  = 0. # 2.398E-04 -> essentially zero
dEdx['H']  = 0. #3.437E-05 -> essentially zero
dEdx['Br'] = 0. #9.814E-04 -> essentially zero
dEdx['W']  = 2.210
#-------- 

dEdx['Copper'] = 1.257
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#1996
dEdx['H_Scintillator'] = 0.91512109*dEdx['C'] + 0.084878906*dEdx['H']
dEdx['Silicon'] = 0.3876
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#2730
dEdx['M_NEMA_FR4_plate'] = 0.18077359*dEdx['Silicon'] + 0.4056325*dEdx['0'] + 0.27804208*dEdx['C'] + 0.068442752*dEdx['H'] + 0.067109079*dEdx['Br']
dEdx['Other'] = 0.
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#0290
dEdx['Air'] = 0.
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#3692
dEdx['StainlessSteel'] = 0.6996*dEdx['Fe']+0.01*dEdx['Mn']+0.19*dEdx['Cr']+0.1*dEdx['Ni']+0.0004*dEdx['C'];
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#0568
dEdx['WCu'] = 0.75*dEdx['W']+0.25*dEdx['Copper']
#--------
dEdx['Lead'] = 1.274 #Pb
#Composition of cable as Sunanda uses them is here: 
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#2841
dEdx['Cables'] = 0.586*dEdx['Copper'] + 0.259*dEdx['C'] + 0.138*dEdx['0'] + 0.017*dEdx['H']

#In mm
MatXo = OrderedDict()
MatXo['Copper'] = 14.3559
MatXo['H_Scintillator'] = 425.393
MatXo['Cables'] = 66.722
MatXo['M_NEMA_FR4_plate'] = 175.056
MatXo['Silicon'] = 93.6762
MatXo['Other'] = 0.
MatXo['Air'] = 301522.
MatXo['StainlessSteel'] = 17.3555
MatXo['WCu'] = 5.1225
MatXo['Lead'] = 5.6118

In [4]:
# Let's read the input file
# Mat is prepoint material, Z is post point material start, so 
# upper edge of prepoint material. Etable,Efull is the energy loss in the 
# prepoint volume with GetDEDX and ComputeTotalDEDX.  
matZ = pd.read_csv(inputfile, sep=" ", header=None, names=["Mat", "Z", "Eta", "R", "Etable", "Efull"], index_col=False)

In [5]:
# We will make a new column with the physical thickness of the volumes in mm
matZ["PhysThickInmm"] = abs(matZ["Z"].shift(-1) -  matZ["Z"])
matZ["PhysThickInmm"] = matZ["PhysThickInmm"].shift(1)

In [6]:
# We will add a column that indicates the track the relevant volume belongs to. 
# The logic is that right before the next track "PhysThickInmm" column will be 
# very large. 
matZ["trackflag"] = matZ.apply(lambda row: True if row["PhysThickInmm"] < 2000. else False ,axis=1)
matZ["tracknum"] = (( matZ["trackflag"] == False) & (matZ["trackflag"].shift() == True)).cumsum()
matZ["tracknum"] = matZ["tracknum"].shift(1)
matZ = matZ.drop('trackflag', 1)

In [7]:
# Now that we have the tracknum we will not let the last Copper volume to 
# destroy our calculation with its huge thickness due to track change. 
#According to TDR BH back is at 5137.7 mm so we put 6.0 mm for that Copper
matZ.loc[ matZ["PhysThickInmm"] > 2000. , "PhysThickInmm" ] = 6.0

In [8]:
# Again, adding a new column with the physical thickness of the volumes in radiation lengths
matZ["PhysThickInXo"] = matZ.apply(lambda row: row["PhysThickInmm"] / MatXo[row["Mat"]],axis=1)

In [9]:
# Adding a new column with the dEdx of the material that the volumes is build
matZ["dEdx"] = matZ.apply(lambda row: dEdx[row["Mat"]],axis=1)

In [10]:
# Another column with the dEdx times thickness to help us with the calculation of the 
# final dEdx weights 
matZ["dEdxtimesdx"] = matZ["dEdx"] * matZ["PhysThickInmm"]

In [11]:
# And here a column with the cumulative sum
matZ["dEdxtimesdxCum"] = matZ.groupby('tracknum')["dEdxtimesdx"].cumsum()

In [12]:
# And here a column with the cumulative sum for Etable
matZ["EtableCum"] = matZ.groupby('tracknum')["Etable"].cumsum()

In [13]:
# And here a column with the cumulative sum for Efull
matZ["EfullCum"] = matZ.groupby('tracknum')["Efull"].cumsum()

In [14]:
# And here a column for the cumulative sum in radiation length 
matZ["PhysThickInXoCum"] = matZ.groupby('tracknum')["PhysThickInXo"].cumsum()

In [15]:
# We will add a column that indicates the layer the relevant volume belongs to. 
# The logic is that if the previous material is Silicon or Scintillator
# we change layer.
matZ["layerflag"] = matZ.apply(lambda row: False if row["Mat"] == "Silicon" or row["Mat"] == "H_Scintillator" else True ,axis=1)
matZ["layer"] = ( matZ["layerflag"] == True) & (matZ["layerflag"].shift(1) == False) 
matZ["layer"] = matZ.groupby('tracknum')["layer"].cumsum()
#The convention is that layers starts from 1
matZ["layer"] = matZ.apply(lambda row: row["layer"] + 1,axis=1)

In [16]:
# Drop auxillary columns
#We need layerflag for not counting the silicon/scintillator in the dedx. 
#matZ = matZ.drop('layerflag', 1)

In [17]:
# This was my misunderstanding. There should be 53 in the layers column since 
# after the last scintillator or silicon the index will increase, there is 
# material there. In case we want to filter them out uncomment the following. 
# Be careful! Filter chooses and not disgards!
# matZ = matZ.groupby('tracknum').filter(lambda g: ~(g['layer'] == 53.0).any()  ) 

In [18]:
#display(matZ)
#matZ[(matZ["layer"] == 1) & matZ["tracknum"] == 20]
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#matZ[(matZ["tracknum"] == 6.0)]
matZ = matZ.dropna()
matZ[ matZ['tracknum'] == 1.0]
#matZ[(matZ["Mat"] == "Air")]
#matZ[ (matZ["Z"] > 3950) & (matZ["Z"] < 4000) & (matZ["Mat"] == "Silicon") & (matZ["R"] < 900)]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["tracknum"] == 5) ]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["Z"] < 3900) & (matZ["Z"] > 0) & (matZ["PhysThickInmm"] > 0.22)]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["layer"] > 28) & (matZ["layer"] < 36)& (matZ["PhysThickInmm"] < 0.22)]
#matZ[matZ["layer"] >= 36]
#matZ[(matZ["Mat"] == "WCu") & (matZ["Z"] > 3950) & (matZ["Z"] < 4000)]

,Mat,Z,Eta,R,Etable,Efull,PhysThickInmm,tracknum,PhysThickInXo,dEdx,dEdxtimesdx,dEdxtimesdxCum,EtableCum,EfullCum,PhysThickInXoCum,layerflag,layer
1018,StainlessSteel,-3190.80,-2.26529,668.666,0.358472,0.369481,0.30,1.0,0.017286,1.139861,0.341958,0.341958,0.358472,0.369481,0.017286,True,1.0
1019,Lead,-3192.90,-2.26569,669.107,2.794680,2.871460,2.10,1.0,0.374211,1.274000,2.675400,3.017358,3.153152,3.240941,0.391497,True,1.0
1020,Copper,-3193.00,-2.26568,669.128,0.114805,0.131771,0.10,1.0,0.006966,1.257000,0.125700,3.143058,3.267957,3.372712,0.398463,True,1.0
1021,StainlessSteel,-3193.30,-2.26567,669.191,0.358480,0.369505,0.30,1.0,0.017286,1.139861,0.341958,3.485017,3.626437,3.742217,0.415748,True,1.0
1022,M_NEMA_FR4_plate,-3194.90,-2.26570,669.527,0.320465,0.526305,1.60,1.0,0.009140,0.179950,0.287920,3.772937,3.946902,4.268522,0.424888,True,1.0
1023,Air,-3196.40,-2.26570,669.841,0.000250,0.000351,1.50,1.0,0.000005,0.000000,0.000000,3.772937,3.947152,4.268873,0.424893,True,1.0
1024,M_NEMA_FR4_plate,-3198.00,-2.26557,670.177,0.320467,0.526337,1.60,1.0,0.009140,0.179950,0.287920,4.060857,4.267619,4.795210,0.434033,True,1.0
1025,Silicon,-3198.12,-2.26552,670.202,0.041900,0.050975,0.12,1.0,0.001281,0.387600,0.046512,4.107369,4.309519,4.846185,0.435314,False,1.0
1026,Silicon,-3198.30,-2.26560,670.240,0.062850,0.076462,0.18,1.0,0.001922,0.387600,0.069768,4.177137,4.372370,4.922647,0.437236,False,1.0
1027,WCu,-3199.70,-2.26708,670.534,2.076030,2.745810,1.40,1.0,0.273304,1.971750,2.760450,6.937587,6.448400,7.668457,0.710540,True,2.0


In [19]:
#matZ[matZ["Z"]>0]
#matZ[matZ["PhysThickInmm"]> 2000.]
#matZ[matZ["tracknum"] == 2 ]
#matZ[ (matZ["tracknum"] == 20) & (matZ["layer"] > 10) ]
#matZ[450:500]

In [20]:
# We will add a column with the dedx weights
# First dedxtimesdx sum per layer but not including silicon or scintillator
matZ = matZ.drop(matZ[ (matZ.Mat == "Silicon") | (matZ.Mat == "H_Scintillator") ].index)
# This is with the theoretical dedx
mdEdxtimesdxsumperlayer = matZ.groupby(['tracknum','layer'])["dEdxtimesdx"].sum()
# And now with the detailed simulation
mdEdxtimesdxsumperlayer_detailedtable = matZ.groupby(['tracknum','layer'])["Etable"].sum()
mdEdxtimesdxsumperlayer_detailedfull = matZ.groupby(['tracknum','layer'])["Efull"].sum()

#type(mdEdxtimesdxsumperlayer)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.to_frame().reset_index()
mdEdxtimesdxsumperlayer_detailedtable = mdEdxtimesdxsumperlayer_detailedtable.to_frame().reset_index()
mdEdxtimesdxsumperlayer_detailedfull = mdEdxtimesdxsumperlayer_detailedfull.to_frame().reset_index()

#mdEdxtimesdxsumperlayer
#type(mdEdxtimesdxsumperlayer)

#Let's put also the accumulated energy loss
mdEdxtimesdxsumperlayer["dEdxtimesdxCum"] = mdEdxtimesdxsumperlayer.groupby('tracknum')["dEdxtimesdx"].cumsum()
mdEdxtimesdxsumperlayer_detailedtable["EtableCum"] = mdEdxtimesdxsumperlayer_detailedtable.groupby('tracknum')["Etable"].cumsum()
mdEdxtimesdxsumperlayer_detailedfull["EfullCum"] = mdEdxtimesdxsumperlayer_detailedfull.groupby('tracknum')["Efull"].cumsum()

#Final weights
mdEdxtimesdxsumperlayer["dedxweights"] = (mdEdxtimesdxsumperlayer["dEdxtimesdx"] + mdEdxtimesdxsumperlayer["dEdxtimesdx"].shift(-1))/2
mdEdxtimesdxsumperlayer_detailedtable["dedxweights_detailedsimulationtable"] = (mdEdxtimesdxsumperlayer_detailedtable["Etable"] + mdEdxtimesdxsumperlayer_detailedtable["Etable"].shift(-1))/2
mdEdxtimesdxsumperlayer_detailedfull["dedxweights_detailedsimulationfull"] = (mdEdxtimesdxsumperlayer_detailedfull["Efull"] + mdEdxtimesdxsumperlayer_detailedfull["Efull"].shift(-1))/2

#Hack for the last layer
mdEdxtimesdxsumperlayer.loc[mdEdxtimesdxsumperlayer["layer"] == 52.0, "dedxweights"] = 89.366024
mdEdxtimesdxsumperlayer_detailedtable.loc[mdEdxtimesdxsumperlayer_detailedtable["layer"] == 52.0, "dedxweights_detailedsimulationtable"] = 91.800143
mdEdxtimesdxsumperlayer_detailedfull.loc[mdEdxtimesdxsumperlayer_detailedfull["layer"] == 52.0, "dedxweights_detailedsimulationfull"] = 98.971647

#Drop duplicates not needed columns
mdEdxtimesdxsumperlayer_detailedtable = mdEdxtimesdxsumperlayer_detailedtable.drop(['tracknum', 'layer'], axis=1)
mdEdxtimesdxsumperlayer_detailedfull = mdEdxtimesdxsumperlayer_detailedfull.drop(['tracknum', 'layer'], axis=1)

#mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ]
#mdEdxtimesdxsumperlayer_detailed[ mdEdxtimesdxsumperlayer_detailed['tracknum'] == 6.0  ]
mdEdxtimesdxsumperlayer = pd.concat([mdEdxtimesdxsumperlayer, mdEdxtimesdxsumperlayer_detailedtable, mdEdxtimesdxsumperlayer_detailedfull], axis=1)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.dropna()

#Adding two columns dedxtable/dedxtheory and dedxfull/dedxtheory
mdEdxtimesdxsumperlayer["EtableoverEtheory"] = mdEdxtimesdxsumperlayer["Etable"]/mdEdxtimesdxsumperlayer["dEdxtimesdx"]
mdEdxtimesdxsumperlayer["EfulloverEtheory"] = mdEdxtimesdxsumperlayer["Efull"]/mdEdxtimesdxsumperlayer["dEdxtimesdx"]
mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 1.0  ].dropna()

,tracknum,layer,dEdxtimesdx,dEdxtimesdxCum,dedxweights,Etable,EtableCum,dedxweights_detailedsimulationtable,Efull,EfullCum,dedxweights_detailedsimulationfull,EtableoverEtheory,EfulloverEtheory
105,1.0,1.0,4.060857,4.060857,8.561878,4.267619,4.267619,7.654235,4.795210,4.795210,9.097840,1.050916,1.180837
106,1.0,2.0,13.062900,17.123757,10.592307,11.040850,15.308469,9.789281,13.400470,18.195680,11.498605,0.845207,1.025842
107,1.0,3.0,8.121714,25.245470,10.592307,8.537712,23.846181,9.790181,9.596740,27.792421,11.503030,1.051221,1.181615
108,1.0,4.0,13.062900,38.308370,10.592307,11.042650,34.888831,9.791512,13.409320,41.201741,11.506499,0.845344,1.026519
109,1.0,5.0,8.121714,46.430084,10.592307,8.540373,43.429204,9.792462,9.603678,50.805418,11.511174,1.051548,1.182469
110,1.0,6.0,13.062900,59.492984,10.592307,11.044550,54.473754,9.793761,13.418670,64.224088,11.514570,0.845490,1.027235
111,1.0,7.0,8.121714,67.614697,10.592307,8.542971,63.016726,9.794736,9.610469,73.834557,11.519355,1.051868,1.183306
112,1.0,8.0,13.062900,80.677597,10.592307,11.046500,74.063226,9.796032,13.428240,87.262797,11.522762,0.845639,1.027968
113,1.0,9.0,8.121714,88.799311,10.592307,8.545564,82.608790,9.796972,9.617284,96.880081,11.527302,1.052187,1.184145
114,1.0,10.0,13.062900,101.862211,10.592307,11.048380,93.657170,9.798266,13.437320,110.317401,11.530703,0.845783,1.028663


In [21]:
mdEdxtimesdxsumperlayer[["layer","EtableoverEtheory","EfulloverEtheory"]][ mdEdxtimesdxsumperlayer['tracknum'] == 1.0  ] 

,layer,EtableoverEtheory,EfulloverEtheory
105,1.0,1.050916,1.180837
106,2.0,0.845207,1.025842
107,3.0,1.051221,1.181615
108,4.0,0.845344,1.026519
109,5.0,1.051548,1.182469
110,6.0,0.845490,1.027235
111,7.0,1.051868,1.183306
112,8.0,0.845639,1.027968
113,9.0,1.052187,1.184145
114,10.0,0.845783,1.028663


In [22]:
mdEdxtimesdxsumperlayer[["layer","dedxweights","dedxweights_detailedsimulationtable","dedxweights_detailedsimulationfull"]][ mdEdxtimesdxsumperlayer['tracknum'] == 1.0  ] 

,layer,dedxweights,dedxweights_detailedsimulationtable,dedxweights_detailedsimulationfull
105,1.0,8.561878,7.654235,9.097840
106,2.0,10.592307,9.789281,11.498605
107,3.0,10.592307,9.790181,11.503030
108,4.0,10.592307,9.791512,11.506499
109,5.0,10.592307,9.792462,11.511174
110,6.0,10.592307,9.793761,11.514570
111,7.0,10.592307,9.794736,11.519355
112,8.0,10.592307,9.796032,11.522762
113,9.0,10.592307,9.796972,11.527302
114,10.0,10.592307,9.798266,11.530703


In [23]:
mdEdxtimesdxsumperlayer[["layer","dEdxtimesdxCum","EtableCum","EfullCum"]][ mdEdxtimesdxsumperlayer['tracknum'] == 1.0  ] 

,layer,dEdxtimesdxCum,EtableCum,EfullCum
105,1.0,4.060857,4.267619,4.795210
106,2.0,17.123757,15.308469,18.195680
107,3.0,25.245470,23.846181,27.792421
108,4.0,38.308370,34.888831,41.201741
109,5.0,46.430084,43.429204,50.805418
110,6.0,59.492984,54.473754,64.224088
111,7.0,67.614697,63.016726,73.834557
112,8.0,80.677597,74.063226,87.262797
113,9.0,88.799311,82.608790,96.880081
114,10.0,101.862211,93.657170,110.317401
